In [19]:

pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [20]:
import numpy as np
import pandas as pd
import torch
import gc; gc.enable()
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.metrics import roc_auc_score
import category_encoders as ce
from sklearn.linear_model import HuberRegressor, RANSACRegressor,QuantileRegressor, TheilSenRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, RidgeCV, SGDRegressor
import warnings; warnings.filterwarnings("ignore")

pd.options.display.max_columns = 50

TRAIN_PATH = "./train.csv"
TEST_PATH = "./test.csv"
#save data after processing
AFTER_PROCESS_TRAIN_PATH = "./after_train.csv"
AFTER_PROCESS_TEST_PATH = "./after_test.csv"

In [21]:
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)
#train_df.head()
#test_df.head()

In [22]:
#print(train_df.isna().sum())
#print(test_df.isna().sum())

In [23]:
#print(train_df['measurement_0'].value_counts())#0-29
#print("================")
#print(train_df['measurement_1'].value_counts())#0-29
#print("================")
#print(train_df['measurement_2'].value_counts())#0-24
#print("================")
#attribute_0~3
#A:7,8,9,5
#B:5,5,8,8
#C:7,8,5,8
#D:7,5,6,6
#E:7,6,6,9
#F:5,6,6,4
#G:5,6,9,7
#H:7,7,7,9
#I:7,5,9,5
#print("================")
#for measurement 3~16 他們的上下界很接近，最大約20，表示這些資料很集中
#for i in range(3,17):
#    print(i, train_df[f'measurement_{i}'].max(), train_df[f'measurement_{i}'].min())
""" 
measurement data range
3 21.499 13.968
4 16.484 8.008
5 21.425 12.073
6 21.543 12.715
7 15.419 7.968
8 23.807 15.217
9 15.412 7.537
10 22.479 9.323
11 25.64 12.461
12 17.663 5.167
13 22.713 10.89
14 22.303 9.14
15 21.626 9.104
16 24.094 9.701
"""
# 1: 3, 5, 6, 8, 11
# 2: 4, 7, 9, 12
# 3: 10, 13, 14, 15, 16

' \nmeasurement data range\n3 21.499 13.968\n4 16.484 8.008\n5 21.425 12.073\n6 21.543 12.715\n7 15.419 7.968\n8 23.807 15.217\n9 15.412 7.537\n10 22.479 9.323\n11 25.64 12.461\n12 17.663 5.167\n13 22.713 10.89\n14 22.303 9.14\n15 21.626 9.104\n16 24.094 9.701\n'

In [24]:
folds_dict = {f'Fold 1': [['A', 'B', 'C'], ['D', 'E']],
               'Fold 2': [['A', 'B', 'D'], ['C', 'E']],
               'Fold 3': [['A', 'B', 'E'], ['C', 'D']],
               'Fold 4': [['A', 'C', 'D'], ['B', 'E']],
               'Fold 5': [['A', 'C', 'E'], ['B', 'D']],
               'Fold 6': [['A', 'D', 'E'], ['B', 'C']],
               'Fold 7': [['B', 'C', 'D'], ['A', 'E']],
               'Fold 8': [['B', 'C', 'E'], ['A', 'D']],
               'Fold 9': [['B', 'D', 'E'], ['A', 'C']],
               'Fold 10': [['C', 'D', 'E'], ['A', 'B']] 
               }

In [25]:
features = ['loading', 'm1', 'm2', 'measurement_17', 'measurement_0', 'measurement_1', 'measurement_2', 
            'measurement_median', 'measurement_max', 'measurement_min', 'measurement_skew']

In [26]:
#print("============== test measurement_17 ==============")

#data = pd.concat([train_df, test_df])

#dro_col = [col for col in test_df.columns if 'measurement' not in col] + ['measurement_0', 'measurement_1', 'measurement_2']
#for x in data.product_code.unique() : #x is the value in A,B,C,D,E,F,G,H,I
#    corr = np.absolute(data[data.product_code == x].drop(dro_col, axis=1).corr()['measurement_17']).sort_values(ascending=False)
#    print(f'{x}:', corr)

"""
0.25/0.2/
A:8,5,6/,7/,4
B:7,4,5/,9,3/
C:8,5,7,9//-,6 - mean it large between 0.2
D:6,5,8,7//,3
E:6,8,5,4//,9
F:6,4,7,5//-,16
G:6,4,8,9//,5
H:5,9,4,8,7//
I:8,3,7/,9/,4
"""

'\n0.25/0.2/\nA:8,5,6/,7/,4\nB:7,4,5/,9,3/\nC:8,5,7,9//-,6 - mean it large between 0.2\nD:6,5,8,7//,3\nE:6,8,5,4//,9\nF:6,4,7,5//-,16\nG:6,4,8,9//,5\nH:5,9,4,8,7//\nI:8,3,7/,9/,4\n'

In [27]:
def related_processing(train, test):
    data = pd.concat([train, test])

    related_dict ={}
    related_dict['measurement_17'] = {
        'A': ['measurement_8','measurement_5','measurement_6','measurement_7'],
        'B': ['measurement_7','measurement_4','measurement_5','measurement_9'],
        'C': ['measurement_8','measurement_5','measurement_7','measurement_9'],
        'D': ['measurement_6','measurement_5','measurement_8','measurement_7'],
        'E': ['measurement_6','measurement_8','measurement_5','measurement_4'],
        'F': ['measurement_6','measurement_4','measurement_7','measurement_5'],
        'G': ['measurement_6','measurement_4','measurement_8','measurement_9'], 
        'H': ['measurement_5','measurement_9','measurement_4','measurement_8'],
        'I': ['measurement_8','measurement_3','measurement_7','measurement_9']
    }
    
    # collect the name of the next 10 best measurement columns sorted by correlation (except 17 already done above):
    dro_col = [col for col in test.columns if 'measurement' not in col] + ['measurement_0', 'measurement_1', 'measurement_2']
    correlation_sum = []
    selected_columns = []
    
    # 看 measurement 彼此間的關係相不相近
    for x in range(3,17):
        #absolute 算絕對值, drop the column in col
        corr = np.absolute(data.drop(dro_col, axis=1).corr()[f'measurement_{x}']).sort_values(ascending=False)
        correlation_sum.append(np.round(np.sum(corr[1:4]),3)) # 取到小數點後第三位
        selected_columns.append(f'measurement_{x}')
    
    correlation_rank = pd.DataFrame()
    correlation_rank['Selected columns'] = selected_columns
    correlation_rank['correlation sum'] = correlation_sum
    correlation_rank = correlation_rank.sort_values(by = 'correlation sum', ascending = False).reset_index(drop = True)
    print(f'Columns selected by the correlation sum : ')
    display(correlation_rank.head(14))

    for i in range(12):
        # from c 取 top 12
        measurement_col = correlation_rank.iloc[i, 0]
        fill_dict = {}

        for x in data.product_code.unique() : # x's value : A,B,C,D,E,F,G,H,I
            # 在 product X 中以 measurement_col 為基準去取 corr
            corr = np.absolute(data[data.product_code == x].drop(dro_col, axis=1).corr()[measurement_col]).sort_values(ascending=False)
            fill_dict[x] = corr[1:5].index.tolist()

        related_dict[measurement_col] = fill_dict
    #related dict -> 對product X的 measurement_? 取四個相近的measurement 參數
    return related_dict

In [28]:
def data_processing(df_train, df_test, related_dict):
    data = pd.concat([df_train, df_test])
    
    feature = [col for col in df_test.columns if col.startswith('measurement') or col=='loading']
    
    #select the column if it has any row is NAN
    null_cols = [col for col in df_train.columns if df_train[col].isnull().sum()!=0]
    
    # according to the full_fill_dict(key, value) to decide the NAN in key through value bt linear model(if value has any NAN)
    # others will decide by KNN model 
    for code in data.product_code.unique(): # A~I
        total_na_filled_by_linear_model = 0
        print(f'\n-------- Product code {code} ----------\n')
        
        for measurement_col in list(related_dict.keys()):
            tmp = data[data.product_code == code]
            related_column = related_dict[measurement_col][code]
            #train the data which column is measurement & other column is not NAN by HuberRegressor
            tmp_train = tmp[related_column+[measurement_col]].dropna(how='any')
            tmp_test = tmp[(tmp[related_column].isnull().sum(axis=1)==0)&(tmp[measurement_col].isnull())]
            
            model = HuberRegressor(epsilon=1.9, max_iter=300)#default //0.59116
            #model = RANSACRegressor()//0.59095
            #model = TheilSenRegressor()//0.59104
            #model = LinearRegression()//0.59112
            #model = Ridge(alpha=4.0)#//0.59122
            #model = RidgeCV()//0.59116
            #model = SGDRegressor()//0.59116
            model.fit(tmp_train[related_column], tmp_train[measurement_col])
            data.loc[(data.product_code==code)&(data[related_column].isnull().sum(axis=1)==0)&(data[measurement_col].isnull()),measurement_col] = model.predict(tmp_test[related_column])
            total_na_filled_by_linear_model += len(tmp_test)
        
        # others NA columns:
        NA = data.loc[data["product_code"] == code,null_cols ].isnull().sum().sum()
        model1 = KNNImputer(n_neighbors=3, weights='uniform')
        data.loc[data.product_code==code, feature] = model1.fit_transform(data.loc[data.product_code==code, feature])
        #KNN only 0.58948
        #model_sim = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
        #data.loc[data.product_code==code, feature] = model_sim.fit_transform(data.loc[data.product_code==code, feature])

        #SimpleImputer mean//0.59031
        #SimpleImputer median//0.59048
        #SimpleImputer most_frequent//0.59103 //only 0.59056
        print(f'{total_na_filled_by_linear_model} filled by linear model ') 
        print(f'{NA} filled by KNN ')


    data['measurement_median'] = data[[f'measurement_{i}' for i in range(3, 17)]].median(axis=1)
    data['measurement_max'] = data[[f'measurement_{i}' for i in range(3, 17)]].max(axis=1)
    data['measurement_min'] = data[[f'measurement_{i}' for i in range(3, 17)]].min(axis=1)
    data['measurement_skew'] = data[[f'measurement_{i}' for i in range(3, 17)]].skew(axis=1)
    
    df_train = data.iloc[:df_train.shape[0],:]
    df_test = data.iloc[df_train.shape[0]:,:]

    # material1 with its size
    encoder = ce.CatBoostEncoder(cols = ['attribute_0', 'attribute_2'])
    encoder.fit(df_train, df_train['failure'])
    df_train = encoder.transform(df_train)
    df_test = encoder.transform(df_test)
    
    # material2 with its size
    encoder = ce.CatBoostEncoder(cols = ['attribute_1', 'attribute_3'])
    encoder.fit(df_train, df_train['failure'])
    df_train = encoder.transform(df_train)
    df_test = encoder.transform(df_test)

    #Target 59121 the same as Catboost

    df_train['m1'] = df_train['attribute_0'] * df_train['attribute_2']
    df_train['m2'] = df_train['attribute_1'] * df_train['attribute_3']
    df_test['m1'] = df_test['attribute_0'] * df_test['attribute_2']
    df_test['m2'] = df_test['attribute_1'] * df_test['attribute_3']

    return df_train, df_test

In [29]:
# data adjust
related_dict = related_processing(train_df, test_df)
df_train, df_test = data_processing(train_df, test_df, related_dict)
# save data after adjust
df_train.to_csv(AFTER_PROCESS_TRAIN_PATH, index=False)
df_test.to_csv(AFTER_PROCESS_TEST_PATH, index=False)

Columns selected by the correlation sum : 


,Selected columns,correlation sum
0,measurement_8,0.454
1,measurement_5,0.386
2,measurement_6,0.365
3,measurement_11,0.342
4,measurement_7,0.336
5,measurement_4,0.331
6,measurement_16,0.252
7,measurement_10,0.244
8,measurement_14,0.225
9,measurement_9,0.201



-------- Product code A ----------

2728 filled by linear model 
1121 filled by KNN 

-------- Product code B ----------

2898 filled by linear model 
1077 filled by KNN 

-------- Product code C ----------

3134 filled by linear model 
1210 filled by KNN 

-------- Product code D ----------

2938 filled by linear model 
1035 filled by KNN 

-------- Product code E ----------

3080 filled by linear model 
1052 filled by KNN 

-------- Product code F ----------

2999 filled by linear model 
1074 filled by KNN 

-------- Product code G ----------

2840 filled by linear model 
1084 filled by KNN 

-------- Product code H ----------

2834 filled by linear model 
1049 filled by KNN 

-------- Product code I ----------

2858 filled by linear model 
971 filled by KNN 


In [30]:
test_predictions = np.zeros((df_test.shape[0], 1))

auc_score = []

for fold in folds_dict.keys():
    
    x_train = df_train[df_train['product_code'].isin(folds_dict[fold][0])][features].values
    y_train = df_train[df_train['product_code'].isin(folds_dict[fold][0])]['failure'].values
    x_valid = df_train[df_train['product_code'].isin(folds_dict[fold][1])][features].values 
    y_valid = df_train[df_train['product_code'].isin(folds_dict[fold][1])]['failure'].values
    
    model = LogisticRegression(max_iter=500, C=0.15, dual=False, penalty="l2", solver='lbfgs')
    model.fit(x_train, y_train)
    
    # save model
    torch.save(model, f'model{fold}.pt')

    # check AUC score
    predictions = model.predict_proba(x_valid)[:, 1].reshape(-1, 1)
    score = roc_auc_score(y_valid, predictions)
    auc_score.append(score)
    print(f'{fold} AUC score : {score}')

#每個fold計算依次最後取平均
print(f'\naverage AUC score: {np.mean(np.array(auc_score))}')

Fold 1 AUC score : 0.5861177210355131
Fold 2 AUC score : 0.5811713672909862
Fold 3 AUC score : 0.5855552332671248
Fold 4 AUC score : 0.5876361402932417
Fold 5 AUC score : 0.5965361011815244
Fold 6 AUC score : 0.5889862795669131
Fold 7 AUC score : 0.5855714221701793
Fold 8 AUC score : 0.5944137441193753
Fold 9 AUC score : 0.5821074931219702
Fold 10 AUC score : 0.590175372731446

average AUC score: 0.5878270874778274


In [31]:
#submission_model_test1.csv C = 0.25
#submission_model_test2.csv C = 0.4
#submission_model_test3.csv C = 0.15 best private score
#submission_model_test3_1.csv C = 0.15 iter=800 not influence
#submission_model_test4.csv C = 0.2
#submission_model_pretest1.csv didn't use 3,5missing
#submission_model_pretest2.csv not 3,5missing & median
#submission_model_pretest3.csv not 3,5missing & add encode attribute_1 & add measurement_max
#submission_model_pretest4.csv not 3,5missing & add encode attribute_1 & with all the measurement->is bad result
#submission_model_pretest5.csv not 3,5missing & add encode attribute_1 & without median
#submission_model_pretest6.csv not 3,5missing & use m1,m2 not (attribute0,1 & area)//0.59018
#submission_model_lbfgs.csv use lbfgs logisticregression model //0.5904
#submission_model_liblinear.csv use liblinear logisticregression model //0.5904
#submission_model_sag.csv use sag logisticregression model //0.59033
#submission_model_saga.csv use saga logisticregression model //0.59019
#submission_model_foldv1.csv use fold_dict & fold_dict2 ,they occupy 70% and 30%
#submission_model_foldv2.csv use fold_dict2
#submission_model_foldv3.csv use fold_dict & fold_dict2 ,they occupy 50% and 50%
#submission_model_foldv4.csv use fold_dict & fold_dict2 ,they occupy 80% and 20%
#submission_model_17v1.csv measurement_17 set 3 related //0.5907
#submission_model_17v2.csv measurement_17 set 4 related //0.59121
#submission_model_17v3.csv measurement_17 set 5 related //0.59086
#submission_model_17v4.csv measurement_17 set related > 0.25 //0.59078
#submission_model_17v5.csv measurement_17 set related > 0.2 //0.59097
#submission_model_17v6.csv measurement_17 set add large from 0.2//0.59121
#submission_model_knnv1.csv knn neighbor = 5 //59109
#submission_model_knnv2.csv knn neighbor = 4 //59117
#submission_model_knnv3.csv knn neighbor = 2 //59089
#submission_model_epv1.csv ep = 1.6 //59114
#submission_model_epv2.csv ep = 1.7 //59119
#submission_model_epv3.csv ep = 1.8 //59121
#submission_model_.csv ep = 1.9 //59121